In [1]:
# Importing standard Qiskit libraries and configuring account
from qiskit import *
from qiskit import IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
style.use('bmh')

from PIL import Image
import math

In [2]:
crop_size = 128
img_dir = "imgs/elephant.png"

data_qb = int(math.log2(crop_size**2))
anc_qb = 1
total_qb = data_qb + anc_qb
print(f"Total Qubits: {total_qb}")

Total Qubits: 15


Crop the image

In [3]:
image = np.array(Image.open(img_dir).convert('L'))
print(f"Original Image Shape: {image.shape}")
original_img = [image.shape[0], image.shape[1]]

image = image.reshape((crop_size, crop_size, int((image.shape[0]/crop_size)**2)))
image = np.transpose(image, (2,0,1))

print(f"Cropped Image Shape: {image.shape}")

Original Image Shape: (512, 512)
Cropped Image Shape: (16, 128, 128)


In [4]:


# Function for plotting the image using matplotlib
def plot_image(img, title: str):
    plt.title(title)
    plt.xticks(range(img.shape[0]))
    plt.yticks(range(img.shape[1]))
    plt.imshow(img, extent=[0, img.shape[0], img.shape[1], 0], cmap='binary')
    plt.show()
    
# plot_image(image, 'Original Image')


In [5]:
# Convert the raw pixel values to probability amplitudes
def amplitude_encode(img_data):
    
    # Calculate the RMS value
    rms = np.sqrt((np.sum(np.square(img_data, dtype='float64'), axis=None)))
    
    # Create normalized image
    image_norm = []
    for arr in img_data:
        for ele in arr:
            image_norm.append(ele / rms)
    # Return the normalized image as a numpy array
    return np.array(image_norm)

# Get the amplitude ancoded pixel values
# Horizontal: Original image
image_norm_h = np.asarray([amplitude_encode(image[i]) for i in range(image.shape[0])])

# Vertical: Transpose of Original image
image_norm_v = np.asarray([amplitude_encode(image[i].T) for i in range(image.shape[0])])

In [6]:
# Initialize the amplitude permutation unitary
D2n_1 = np.roll(np.identity(2**total_qb), 1, axis=1)

In [7]:
def generateCircuit(image_norm_h, image_norm_v):

    # Create the circuit for horizontal scan
    qc_h = QuantumCircuit(total_qb)
    qc_h.initialize(image_norm_h, range(1, total_qb))
    qc_h.h(0)
    qc_h.unitary(D2n_1, range(total_qb))
    qc_h.h(0)
    display(qc_h.draw('mpl', fold=-1))

    # Create the circuit for vertical scan
    qc_v = QuantumCircuit(total_qb)
    qc_v.initialize(image_norm_v, range(1, total_qb))
    qc_v.h(0)
    qc_v.unitary(D2n_1, range(total_qb))
    qc_v.h(0)
    display(qc_v.draw('mpl', fold=-1))

    # Combine both circuits into a single list
    circ_list = [qc_h, qc_v]
    
    return qc_h, qc_v, circ_list

In [8]:
def QuantumSimulate(qc_h, qc_v, circ_list):
    # Simulating the cirucits
    back = Aer.get_backend('statevector_simulator')
    results = execute(circ_list, backend=back).result()
    sv_h = results.get_statevector(qc_h)
    sv_v = results.get_statevector(qc_v)

    from qiskit.visualization import array_to_latex
    print('Horizontal scan statevector:')
    # display(array_to_latex(sv_h[:30], max_size=30))
    print()
    print('Vertical scan statevector:')
    # display(array_to_latex(sv_v[:30], max_size=30))
    
    return sv_h, sv_v

In [9]:
def scanEdge(sv_h, sv_v):
    edge_scan_h = np.array([sv_h[f'{2*i+1:03b}'].real for i in range(2**data_qb)]).reshape(image.shape[0], image.shape[1])
    edge_scan_v = np.array([sv_v[f'{2*i+1:03b}'].real for i in range(2**data_qb)]).reshape(image.shape[0], image.shape[1]).T

    return edge_scan_h, edge_scan_v

In [10]:
Total_scan_h = []
Total_scan_v = []
for i in range(image_norm_h.shape[0]):
    print(f"Idx: {i}")
    qc_h, qc_v, circ_list = generateCircuit(image_norm_h[i], image_norm_v[i])
    sv_h, sv_v = QuantumSimulate(qc_h, qc_v, circ_list)
    edge_scan_h, edge_scan_v = scanEdge(sv_h, sv_v)
    Total_scan_h.append(edge_scan_h) #[16, 128, 128]
    Total_scan_v.append(edge_scan_v)

Total_scan_h = np.asarray(Total_scan_h)   
Total_scan_v = np.asarray(Total_scan_v)
# Total_scan_h = np.transpose(Total_scan_h, (1,2,0)).reshape(original_img[0], original_img[1])  #[128, 128, 16] -> [512, 512]
# Total_scan_v = np.transpose(Total_scan_v, (1,2,0)).reshape(original_img[0], original_img[1])  #[128, 128, 16] -> [512, 512]

Idx: 0


MemoryError: Unable to allocate 16.0 GiB for an array with shape (32768, 32768) and data type complex128

In [ ]:
# Classical postprocessing for plotting the output

# Defining a lambda function for
# thresholding to binary values
# thr = 2e-20
# threshold = lambda amp: (amp > thr or amp < -thr)

# Selecting odd states from the raw statevector and
# reshaping column vector of size 64 to an 8x8 matrix
# edge_scan_h = np.abs(np.array([1 if threshold(sv_h[2*i+1].real) else 0 for i in range(2**data_qb)])).reshape(image.shape[0], image.shape[1])
# edge_scan_v = np.abs(np.array([1 if threshold(sv_v[2*i+1].real) else 0 for i in range(2**data_qb)])).reshape(image.shape[0], image.shape[1]).T


# Plotting the Horizontal and vertical scans
# plot_image(edge_scan_h, 'Horizontal scan output')
# plot_image(edge_scan_v, 'Vertical scan output')

# plot_image(edge_scan_h, 'Horizontal scan output')
# plot_image(edge_scan_v, 'Vertical scan output')

In [ ]:
# Combining the horizontal and vertical component of the result
# edge_scan_sim = edge_scan_h  | edge_scan_v
# edge_scan_sim = edge_scan_h*0.5 + edge_scan_v*0.5

# # Plotting the original and edge-detected images
# plot_image(image, 'Original image')
# plot_image(edge_scan_sim, 'Edge Detected image')